In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile

import datetime 
base = datetime.date(2020,2,1)
for i in range(0,10): 
    d=base + datetime.timedelta(i)
    d=d.strftime("%Y%m%d")
    print(f"N029kakutokuruiseki{d}.csv")

from datetime import datetime, timedelta

def date_range(start_date: datetime, end_date: datetime):
    diff = (end_date - start_date).days + 1
    return (start_date + timedelta(i) for i in range(diff))

In [2]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [3]:
input_file = "【C063】新規入会者_3月末時点.xlsx"
output_file = "集計_新規入会者_3月末時点"
data_dir = Path("C:/Users/yakad\Documents/Yusuke_Akada_Jupyter_Notebook-master/file")

filename = input_file
print(filename)

【C063】新規入会者_3月末時点.xlsx


column_names = ['実績月','入会申込番号','直近３ケ月買物利用金額','直近３ケ月全体利用金額','年令','性別',
                '都道府県名','顧客状況コ－ド','マイペ設定','DM希望店','カード券種名','カ－ド加入日','初回利用日',
                'あとからリボ申込有無コード','Weｂ明細有無コード','解約','入会チャネル']

In [5]:
df = pd.read_excel(data_dir/f"{filename}")
df.head()

,実績月,入会申込番号,直近３ケ月買物利用金額,直近３ケ月全体利用金額,年令,性別,都道府県名,顧客状況コ－ド,マイペ設定,DM希望店,カード券種名,カ－ド加入日,初回利用日,あとからリボ申込有無コード,Weｂ明細有無コード,解約,入会チャネル
0,201910,155051921422628,910,910,77,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,20190801,20190816,1,1,0,オンライン
1,201910,155051921427056,910,910,77,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,20190801,20190816,1,1,0,オンライン
2,201910,155051921427064,910,910,77,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,20190801,20190816,1,1,0,オンライン
3,201910,155193016401040,594,594,52,女性,大阪府,0,0,Ｄ_心斎橋店,大丸松坂屋VISAゴールド,20190801,20190918,0,1,0,店頭タブレット
4,201910,155193006218453,151,151,46,女性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,20190802,20190818,0,0,0,店頭タブレット


In [6]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(63197, 17)
RangeIndex(start=0, stop=63197, step=1)
Index(['実績月', '入会申込番号', '直近３ケ月買物利用金額', '直近３ケ月全体利用金額', '年令', '性別', '都道府県名',
       '顧客状況コ－ド', 'マイペ設定', 'DM希望店', 'カード券種名', 'カ－ド加入日', '初回利用日',
       'あとからリボ申込有無コード', 'Weｂ明細有無コード', '解約', '入会チャネル'],
      dtype='object')
実績月               int64
入会申込番号            int64
直近３ケ月買物利用金額       int64
直近３ケ月全体利用金額       int64
年令                int64
性別               object
都道府県名            object
顧客状況コ－ド          object
マイペ設定             int64
DM希望店            object
カード券種名           object
カ－ド加入日            int64
初回利用日             int64
あとからリボ申込有無コード     int64
Weｂ明細有無コード        int64
解約                int64
入会チャネル           object
dtype: object


In [7]:
df['カ－ド加入日'] = pd.to_datetime(df['カ－ド加入日'], format = '%Y%m%d')
df['カ－ド加入年'] = df['カ－ド加入日'].dt.year
df['カ－ド加入月'] = df['カ－ド加入日'].dt.month
df.head()

,実績月,入会申込番号,直近３ケ月買物利用金額,直近３ケ月全体利用金額,年令,性別,都道府県名,顧客状況コ－ド,マイペ設定,DM希望店,カード券種名,カ－ド加入日,初回利用日,あとからリボ申込有無コード,Weｂ明細有無コード,解約,入会チャネル,カ－ド加入年,カ－ド加入月
0,201910,155051921422628,910,910,77,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,2019-08-01,20190816,1,1,0,オンライン,2019,8
1,201910,155051921427056,910,910,77,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,2019-08-01,20190816,1,1,0,オンライン,2019,8
2,201910,155051921427064,910,910,77,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,2019-08-01,20190816,1,1,0,オンライン,2019,8
3,201910,155193016401040,594,594,52,女性,大阪府,0,0,Ｄ_心斎橋店,大丸松坂屋VISAゴールド,2019-08-01,20190918,0,1,0,店頭タブレット,2019,8
4,201910,155193006218453,151,151,46,女性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,2019-08-02,20190818,0,0,0,店頭タブレット,2019,8


In [8]:
df['入会チャネル'].unique()

array(['オンライン', '店頭タブレット', '郵送'], dtype=object)

df[df['入会チャネル']=='オンライン'].head()

In [9]:
df['実績月'].unique()

array([201910, 202003], dtype=int64)

df = df[df['実績月'] == 202003]
print(df.shape)

In [10]:
#入会申込番号の重複がないかチェック
print(len(df) == len(df['入会申込番号'].unique()))

False


In [60]:
#重複削除
df_dup = df.drop_duplicates(subset=['入会申込番号','実績月'])

In [21]:
df_dup.shape[0]

54720

In [13]:
#欠損データがないかチェック
df_dup.isnull().sum()

実績月              0
入会申込番号           0
直近３ケ月買物利用金額      0
直近３ケ月全体利用金額      0
年令               0
性別               0
都道府県名            0
顧客状況コ－ド          0
マイペ設定            0
DM希望店            0
カード券種名           0
カ－ド加入日           0
初回利用日            0
あとからリボ申込有無コード    0
Weｂ明細有無コード       0
解約               0
入会チャネル           0
カ－ド加入年           0
カ－ド加入月           0
dtype: int64

In [61]:
#SettingWithCopyWarning回避対策
df_dup2 = df_dup.drop(columns=['あとからリボ申込有無コード'])
df_dup2['初回利用日フラグ'] = df_dup2['初回利用日'].apply(lambda x : "1" if x > 0 else "0")

In [62]:
#利用無し客を置き換え
df_dup2.loc[df_dup2['初回利用日'] <= 0, '初回利用日'] = 20301231
df_dup2.head()

,実績月,入会申込番号,直近３ケ月買物利用金額,直近３ケ月全体利用金額,年令,性別,都道府県名,顧客状況コ－ド,マイペ設定,DM希望店,カード券種名,カ－ド加入日,初回利用日,Weｂ明細有無コード,解約,入会チャネル,カ－ド加入年,カ－ド加入月,初回利用日フラグ
0,201910,155051921422628,910,910,77,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,2019-08-01,20190816,1,0,オンライン,2019,8,1
1,201910,155051921427056,910,910,77,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,2019-08-01,20190816,1,0,オンライン,2019,8,1
2,201910,155051921427064,910,910,77,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,2019-08-01,20190816,1,0,オンライン,2019,8,1
3,201910,155193016401040,594,594,52,女性,大阪府,0,0,Ｄ_心斎橋店,大丸松坂屋VISAゴールド,2019-08-01,20190918,1,0,店頭タブレット,2019,8,1
4,201910,155193006218453,151,151,46,女性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,2019-08-02,20190818,0,0,店頭タブレット,2019,8,1


#df_dup[['カ－ド加入日','初回利用日']] = df_dup[['カ－ド加入日','初回利用日']].applymap(str).applymap(lambda s: "{}/{}/{}".format(s[0:4],s[4:6],s[6:]))

#df_dup['カ－ド加入日'] = pd.to_datetime(df_dup['カ－ド加入日'], format='%Y-%m-%d')

In [63]:
df_dup2['初回利用日'] = pd.to_datetime(df_dup2['初回利用日'], format = '%Y%m%d').copy()
df_dup2.head()

,実績月,入会申込番号,直近３ケ月買物利用金額,直近３ケ月全体利用金額,年令,性別,都道府県名,顧客状況コ－ド,マイペ設定,DM希望店,カード券種名,カ－ド加入日,初回利用日,Weｂ明細有無コード,解約,入会チャネル,カ－ド加入年,カ－ド加入月,初回利用日フラグ
0,201910,155051921422628,910,910,77,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,2019-08-01,2019-08-16,1,0,オンライン,2019,8,1
1,201910,155051921427056,910,910,77,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,2019-08-01,2019-08-16,1,0,オンライン,2019,8,1
2,201910,155051921427064,910,910,77,男性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,2019-08-01,2019-08-16,1,0,オンライン,2019,8,1
3,201910,155193016401040,594,594,52,女性,大阪府,0,0,Ｄ_心斎橋店,大丸松坂屋VISAゴールド,2019-08-01,2019-09-18,1,0,店頭タブレット,2019,8,1
4,201910,155193006218453,151,151,46,女性,兵庫県,0,0,Ｄ_神戸店,大丸松坂屋VISAゴールド,2019-08-02,2019-08-18,0,0,店頭タブレット,2019,8,1


df_dup['実績月'] = df_dup['実績月'] * 100 + 1

df_dup['実績月'] = pd.to_datetime(df_dup['実績月'], format = '%Y%m%d')

df_dup[['実績月2']] = df_dup[['実績月']].applymap(str).applymap(lambda s: "{}/{}/{}".format(s[0:4],s[4:6],s[6:]))

df_dup['実績月2'] = pd.to_datetime(df_dup['実績月2'], format='%Y-%m-%d')

df_dup = df_dup.drop(columns=['実績月'])

集計

In [64]:
#客数
df_count = df_dup2.groupby(['実績月','入会チャネル','カ－ド加入年','カ－ド加入月','初回利用日フラグ'])['入会申込番号'].count()

In [65]:
#金額
df_amount= df_dup2.groupby(['実績月','入会チャネル','カ－ド加入年','カ－ド加入月','初回利用日フラグ'])['直近３ケ月全体利用金額'].sum()

In [66]:
df_merge = pd.merge(df_count, df_amount, on=['実績月','入会チャネル','カ－ド加入年','カ－ド加入月','初回利用日フラグ'], how='inner')
df_merge.head()

入会申込番号  直近３ケ月全体利用金額
実績月    入会チャネル カ－ド加入年 カ－ド加入月 初回利用日フラグ                     
201910 オンライン  2019   8      0             37         1768
                            1             90        19186
                     9      0             60           80
                            1             81        12319
                     10     0            133            0

In [372]:
df_merge.to_csv("C:/Users/yakad\Documents/Yusuke_Akada_Jupyter_Notebook-master/file/shukei.csv", header=True,encoding='utf-8-sig')

In [ ]:
#アフィリエイト先別に実績集計